In [1]:
import numpy as np
from bitstring import Bits
from bitstring import BitArray
import pickle
import cv2
import time

In [17]:
def Quantization2d(data_arr, min_q, max_q, scale, offset):
	row, col = data_arr.shape
	data_quantization = np.ones((row, col))

	for i in range(row):
		for j in range(col):
			data_quantization[i][j] = round((data_arr[i][j] / scale) + offset)

			if(data_quantization[i][j] > max_q):
				data_quantization[i][j] = max_q
			elif(data_quantization[i][j] < min_q):
				data_quantization[i][j] = min_q

	return data_quantization

def Quantization3d(data, min_q, max_q, scale, offset):
	row, col, num_filter = data.shape
	data_quantization = np.ones((row, col, num_filter))

	for i in range(num_filter):
		data_quantization[:, :, i] = Quantization2d(data[:, :, i], min_q, max_q, scale, offset)
        
	return data_quantization

def Quantization4d(data, min_q, max_q, scale, offset):
	row, col, num_subfilter, num_filter = data.shape
	data_quantization = np.ones((row, col, num_subfilter, num_filter))

	for i in range(num_filter):
		data_quantization[:, :, :, i] = Quantization3d(data[:, :, :, i], min_q, max_q, scale, offset)

	return data_quantization

def Dequantization2d(data_arr, scale, offset):
	row, col = data_arr.shape
	data_dequantization = np.ones((row, col))

	for i in range(row):
		for j in range(col):
			data_dequantization[i][j] = (data_arr[i][j] - offset) * scale

	return data_dequantization

def	Round_quantization2d(data_arr, min_q, max_q):
	row, col = data_arr.shape
	data_round_quantization = np.ones((row, col))

	for i in range(row):
		for j in range(col):
			data = float(data_arr[i][j])
			data_round_quantization[i][j] = round(data)

			if(data_round_quantization[i][j] > max_q):
				data_round_quantization[i][j] = max_q
			elif(data_round_quantization[i][j] < min_q):
				data_round_quantization[i][j] = min_q

	return data_round_quantization

def	Round_quantization3d(data, min_q, max_q):
	row, col, num_filter = data.shape
	data_round_quantization = np.ones((row, col, num_filter))

	for i in range(num_filter):
		data_round_quantization[:, :, i] = Round_quantization2d(data[:, :, i], min_q, max_q)

	return data_round_quantization

In [18]:
def convert_int_to_binary(number, bit_width):
    if((bit_width%4) != 0):
        return str(Bits(int=number, length=bit_width))
    else:
        temp =  str(Bits(int=number, length=bit_width+1))
        return '0b' + temp[-(len(temp)-3):]

def convert_binary_to_hex(number, bit_width):
	if(bit_width <= 4):
		return hex(int(number, 2))
	else:
		number = number[2:len(number)]
		data_ret = ''
		index = np.arange(bit_width, 1, -4)
		for i in range(len(index)):
			if(index[i] >= 4):
				temp = hex(int(number[index[i] - 4 : index[i]], 2))
				data_ret = temp[-1:] + data_ret
			else:
				temp = hex(int(number[0 : index[i]], 2))
				data_ret = temp[-1:] + data_ret
		
		return '0x' + data_ret

def Load_Conv_and_W_and_Bias(data_path, load_option, conv_shape):
	temp_data = np.loadtxt(data_path)

	if(load_option == 0): # ================== Conv
		row, col, num_sub_filter, num_filter = conv_shape # row = col
		data = np.zeros((row, col, num_sub_filter, num_filter))

		index = 0 # maximum index = num_sub_filter * num_filter
		for m in range(num_filter):
			for k in range(num_sub_filter):
				data[:, :, k, m] = temp_data[index * row: index * row + 5, :]
				index = index + 1
		return data
	elif(load_option == 1): # ================ W
		return temp_data
	else: # ============================= Bias
		return temp_data.reshape(-1, 1)

def Load_data_matrix(data_path):
	data = np.loadtxt(data_path)
	return data

def create_txt_file_for_input_verilog(input_data, bit_width, file_path):
    verilog_file = open(file_path, mode = 'w')

    input_data = input_data.reshape(-1, 1)

    num_node, temp = input_data.shape

    for i in range(num_node):
        dec_data = int(input_data[i])
        bin_data_temp = convert_int_to_binary(dec_data, bit_width)
        bin_data = bin_data_temp[-bit_width:]
        hex_data = convert_binary_to_hex('0b' + bin_data, bit_width)
        if(i != (num_node - 1)):
            verilog_file.write(str(hex_data[-(len(hex_data)-2):]) + '\n')
        else:
            verilog_file.write(str(hex_data[-(len(hex_data)-2):]))
    verilog_file.close()

In [19]:
max_image = 1
min_image = -1

max_filter_conv0 = 0.64
min_filter_conv0 = -0.64
max_filter_conv1 = 0.94
min_filter_conv1 = -0.94

max_result_conv0 = 2.67
min_result_conv0 = -2.67
max_result_conv1 = 8.88
min_result_conv1 = -8.88

max_bias_conv0 = 0.35
min_bias_conv0 = -0.35
max_bias_conv1 = 0.16
min_bias_conv1 = -0.16

max_pooling0 = 1.75
min_pooling0 = -1.75
max_pooling1 = 2.84
min_pooling1 = -2.84

max_w0 = 1.02
min_w0 = -1.02
max_w1 = 0.78
min_w1 = -0.78
max_w2 = 0.85
min_w2 = -0.85

max_b0 = 0.21
min_b0 = -0.21
max_b1 = 0.19
min_b1 = -0.19
max_b2 = 0.24
min_b2 = -0.24

max_in = 2.84
min_in = -2.84
max_hd0 = 9.6
min_hd0 = -9.6
max_hd1 = 12.8
min_hd1 = -12.8
max_out_bf_act = 23.4
min_out_bf_act = -23.4
max_out = 1
min_out = -1

In [20]:
# ============================================ Bit-width ============================================
bit_width_image = 6
bit_width_filter_conv0 = 6
bit_width_filter_conv1 = 6
bit_width_conv0 = 6
bit_width_conv1 = 6
bit_width_bias_conv0 = 6
bit_width_bias_conv1 = 6
bit_width_pooling0 = 6
bit_width_pooling1 = 6
bit_width_input = bit_width_pooling1

bit_width_w0 = 6
bit_width_w1 = 6
bit_width_w2 = 6

bit_width_b0 = 6
bit_width_b1 = 6
bit_width_b2 = 6

bit_width_h0 = 6
bit_width_h1 = 6
bit_width_bf_sigmoid = 6
bit_width_output = 6

In [21]:
# ============================================ Max-min quantization value ============================================
max_image_q = 2**(bit_width_image-1)-1
min_image_q = -2**(bit_width_image-1)
max_filter_conv0_q = 2**(bit_width_filter_conv0-1)-1
min_filter_conv0_q = -2**(bit_width_filter_conv0-1)
max_filter_conv1_q = 2**(bit_width_filter_conv1-1)-1
min_filter_conv1_q = -2**(bit_width_filter_conv1-1)
max_conv0_q = 2**(bit_width_conv0-1)-1
min_conv0_q = -2**(bit_width_conv0-1)
max_conv1_q = 2**(bit_width_conv1-1)-1
min_conv1_q = -2**(bit_width_conv1-1)
max_bias_conv0_q = 2**(bit_width_bias_conv0-1)-1
min_bias_conv0_q = -2**(bit_width_bias_conv0-1)
max_bias_conv1_q = 2**(bit_width_bias_conv1-1)-1
min_bias_conv1_q = -2**(bit_width_bias_conv1-1)
max_pooling0_q = 2**(bit_width_pooling0-1)-1
min_pooling0_q = -2**(bit_width_pooling0-1)
max_pooling1_q = 2**(bit_width_pooling1-1)-1
min_pooling1_q = -2**(bit_width_pooling1-1)

max_w0_q = 2**(bit_width_w0-1)-1
min_w0_q = -2**(bit_width_w0-1)
max_w1_q = 2**(bit_width_w1-1)-1
min_w1_q = -2**(bit_width_w1-1)
max_w2_q = 2**(bit_width_w2-1)-1
min_w2_q = -2**(bit_width_w2-1)

max_b0_q = 2**(bit_width_b0-1)-1
min_b0_q = -2**(bit_width_b0-1)
max_b1_q = 2**(bit_width_b1-1)-1
min_b1_q = -2**(bit_width_b1-1)
max_b2_q = 2**(bit_width_b2-1)-1
min_b2_q = -2**(bit_width_b2-1)

max_input_q = 2**(bit_width_input-1)-1
min_input_q = -2**(bit_width_input-1)
max_hd0_q = 2**(bit_width_h0-1)-1
min_hd0_q = -2**(bit_width_h0-1)
max_hd1_q = 2**(bit_width_h1-1)-1
min_hd1_q = -2**(bit_width_h1-1)
max_bf_sigmoid_q = 2**(bit_width_bf_sigmoid-1)-1
min_bf_sigmoid_q = -2**(bit_width_bf_sigmoid-1)
max_output_q = 2**(bit_width_output-1)-1
min_output_q = -2**(bit_width_output-1)

In [22]:
# ============================================ Scale and offset for w1 & bias ============================================
# Quantied number = (X/Scale) + Offset
scale_image = (max_image - min_image)/(max_image_q - min_image_q)
scale_filter_conv0 = (max_filter_conv0 - min_filter_conv0)/(max_filter_conv0_q - min_filter_conv0_q)
scale_filter_conv1 = (max_filter_conv1 - min_filter_conv1)/(max_filter_conv1_q - min_filter_conv1_q)
scale_conv0 = (max_result_conv0 - min_result_conv0)/(max_conv0_q - min_conv0_q)
scale_conv1 = (max_result_conv1 - min_result_conv1)/(max_conv1_q - min_conv1_q)
scale_bias_conv0 = (max_bias_conv0 - min_bias_conv0)/(max_bias_conv0_q - min_bias_conv0_q)
scale_bias_conv1 = (max_bias_conv1 - min_bias_conv1)/(max_bias_conv1_q - min_bias_conv1_q)
scale_pooling0 = (max_pooling0 - min_pooling0)/(max_pooling0_q - min_pooling0_q)
scale_pooling1 = (max_pooling1 - min_pooling1)/(max_pooling1_q - min_pooling1_q)

scale_w0 = (max_w0 - min_w0)/(max_w0_q - min_w0_q)
scale_w1 = (max_w1 - min_w1)/(max_w1_q - min_w1_q)
scale_w2 = (max_w2 - min_w2)/(max_w2_q - min_w2_q)

scale_b0 = (max_b0 - min_b0)/(max_b0_q - min_b0_q)
scale_b1 = (max_b1 - min_b1)/(max_b1_q - min_b1_q)
scale_b2 = (max_b2 - min_b2)/(max_b2_q - min_b2_q)

scale_input = (max_in - min_in)/(max_input_q - min_input_q)
scale_hd0 = (max_hd0 - min_hd0)/(max_hd0_q - min_hd0_q)
scale_hd1 = (max_hd1 - min_hd1)/(max_hd1_q - min_hd1_q)
scale_bf_sigmoid = (max_out_bf_act - min_out_bf_act)/(max_bf_sigmoid_q - min_bf_sigmoid_q)
scale_output = (max_out - min_out)/(max_output_q - min_output_q)

offset_image = round((max_image*min_image_q - min_image*max_image_q)/(max_image - min_image))
offset_filter_conv0 = round((max_filter_conv0*min_filter_conv0_q - min_filter_conv0*max_filter_conv0_q)/(max_filter_conv0 - min_filter_conv0))
offset_filter_conv1 = round((max_filter_conv1*min_filter_conv1_q - min_filter_conv1*max_filter_conv1_q)/(max_filter_conv1 - min_filter_conv1))
offset_conv0 = round((max_result_conv0*min_conv0_q - min_result_conv0*max_conv0_q)/(max_result_conv0 - min_result_conv0))
offset_conv1 = round((max_result_conv1*min_conv1_q - min_result_conv1*max_conv1_q)/(max_result_conv1 - min_result_conv1))
offset_bias_conv0 = round((max_bias_conv0*min_bias_conv0_q - min_bias_conv0*max_bias_conv0_q)/(max_bias_conv0 - min_bias_conv0))
offset_bias_conv1 = round((max_bias_conv1*min_bias_conv1_q - min_bias_conv1*max_bias_conv1_q)/(max_bias_conv1 - min_bias_conv1))
offset_pooling0 = round((max_pooling0*min_pooling0_q - min_pooling0*max_pooling0_q)/(max_pooling0 - min_pooling0))
offset_pooling1 = round((max_pooling1*min_pooling1_q - min_pooling1*max_pooling1_q)/(max_pooling1 - min_pooling1))

offset_w0 = round((max_w0*min_w0_q - min_w0*max_w0_q)/(max_w0 - min_w0))
offset_w1 = round((max_w1*min_w1_q - min_w1*max_w1_q)/(max_w1 - min_w1))
offset_w2 = round((max_w2*min_w2_q - min_w2*max_w2_q)/(max_w2 - min_w2))

offset_b0 = round((max_b0*min_b0_q - min_b0*max_b0_q)/(max_b0 - min_b0))
offset_b1 = round((max_b1*min_b1_q - min_b1*max_b1_q)/(max_b1 - min_b1))
offset_b2 = round((max_b2*min_b2_q - min_b2*max_b2_q)/(max_b2 - min_b2))

offset_input = round((max_in*min_input_q - min_in*max_input_q)/(max_in - min_in))
offset_hd0 = round((max_hd0*min_hd0_q - min_hd0*max_hd0_q)/(max_hd0 - min_hd0))
offset_hd1 = round((max_hd1*min_hd1_q - min_hd1*max_hd1_q)/(max_hd1 - min_hd1))
offset_bf_sigmoid = round((max_out_bf_act*min_bf_sigmoid_q - min_out_bf_act*max_bf_sigmoid_q)/(max_out_bf_act - min_out_bf_act))
offset_output = round((max_out*min_output_q - min_out*max_output_q)/(max_out - min_out))

# ============================================ Scale for quantize ============================================
# X * Scale 
scale_calculate_convolution_0 = (scale_image*scale_filter_conv0)/scale_conv0
scale_for_pooling0 = scale_conv0/scale_pooling0
scale_calculate_convolution_1 = (scale_pooling0*scale_filter_conv1)/scale_conv1
scale_for_pooling1 = scale_conv1/scale_pooling1

scale_for_W0_x_Flatten = (scale_input*scale_w0)/scale_hd0
scale_for_W1_x_hd0 = (scale_hd0*scale_w1)/scale_hd1
scale_for_W2_x_hd1 = (scale_hd1*scale_w2)/scale_bf_sigmoid

scale_for_B0_conv = scale_bias_conv0/scale_conv0
scale_for_B1_conv = scale_bias_conv1/scale_conv1

scale_for_B0_fc = scale_b0/scale_hd0
scale_for_B1_fc = scale_b1/scale_hd1
scale_for_B2_fc = scale_b2/scale_bf_sigmoid

In [23]:
Conv0_shape = (5, 5, 1, 6)
Conv1_shape = (5, 5, 6, 16)

image_path_verilog = '../../Weight_and_bias_of_model_verilog/image.txt'
Conv0_path_verilog = '../../Weight_and_bias_of_model_python/Conv0.txt'
Bias_conv0_path_verilog = '../../Weight_and_bias_of_model_python/Bias_conv0.txt'
Conv1_path_verilog = '../../Weight_and_bias_of_model_python/Conv1.txt'
Bias_conv1_path_verilog = '../../Weight_and_bias_of_model_python/Bias_conv1.txt'

Conv0_path = '../../Weight_and_bias_of_model_python/Conv0.txt'
Bias_conv0_path = '../../Weight_and_bias_of_model_python/Bias_conv0.txt'
Conv1_path = '../../Weight_and_bias_of_model_python/Conv1.txt'
Bias_conv1_path = '../../Weight_and_bias_of_model_python/Bias_conv1.txt'

W0_path = '../../Weight_and_bias_of_model_python/W0.txt'
W1_path = '../../Weight_and_bias_of_model_python/W1.txt'
W2_path = '../../Weight_and_bias_of_model_python/W2.txt'

B0_path = '../../Weight_and_bias_of_model_python/B0.txt'
B1_path = '../../Weight_and_bias_of_model_python/B1.txt'
B2_path = '../../Weight_and_bias_of_model_python/B2.txt'

In [39]:
pickle_in = open("../../Dataset/Data_test_10000.pickle","rb")
image_test = pickle.load(pickle_in)
pickle_in = open("../../Dataset/Label_test_10000.pickle","rb")
label_test = pickle.load(pickle_in)

Conv0 = Load_Conv_and_W_and_Bias(Conv0_path, 1, 0)
Bias_conv0 = Load_Conv_and_W_and_Bias(Bias_conv0_path, 2, 0)
Conv1 = Load_Conv_and_W_and_Bias(Conv1_path, 1, 0)
Bias_conv1 = Load_Conv_and_W_and_Bias(Bias_conv1_path, 2, 0)

Conv1_temp = Load_Conv_and_W_and_Bias(Conv1_path, 0, Conv1_shape)

W0 = Load_Conv_and_W_and_Bias(W0_path, 1, 0)
W1 = Load_Conv_and_W_and_Bias(W1_path, 1, 0)
W2 = Load_Conv_and_W_and_Bias(W2_path, 1, 0)

B0 = Load_Conv_and_W_and_Bias(B0_path, 2, 0)
B1 = Load_Conv_and_W_and_Bias(B1_path, 2, 0)
B2 = Load_Conv_and_W_and_Bias(B2_path, 2, 0)

In [40]:
print(Conv0.shape)
print(Bias_conv0.shape)
print(Conv1.shape)
print(Bias_conv1.shape)
print(Conv1_temp.shape)

(30, 5)
(6, 1)
(480, 5)
(16, 1)
(5, 5, 6, 16)


In [41]:
print(Conv1_temp[:, :, 0, 0])
print('\n==============\n')
print(Conv1_temp[:, :, 1, 0])
print('\n==============\n')
print(Conv1.reshape(-1, 1))

[[ 0.04851859 -0.1438246  -0.54313666 -0.36861646  0.03476332]
 [ 0.17254019 -0.06478588 -0.08988825  0.10441433  0.19314076]
 [ 0.24737994 -0.06551661 -0.23020874 -0.0156035  -0.00678222]
 [-0.04441087 -0.09678386 -0.14068176  0.07402343 -0.11665808]
 [ 0.08713108  0.00606869  0.03400352  0.14854686 -0.18251422]]


[[ 0.0213694   0.09157427  0.19397451  0.06696144  0.08213645]
 [-0.08692447 -0.22725967 -0.22914405 -0.11691716 -0.30679813]
 [ 0.01664766  0.08855011 -0.20456885 -0.02261611 -0.1619465 ]
 [ 0.34814632  0.01951889  0.13625239  0.02631405 -0.2701475 ]
 [ 0.05840284 -0.10904247 -0.2699256  -0.6881372  -0.14021902]]


[[ 0.04851859]
 [-0.1438246 ]
 [-0.54313666]
 ...
 [-0.03556796]
 [-0.43978828]
 [-0.68106097]]


In [ ]:
Conv0 = Quantization2d(Conv0, min_filter_conv0_q, max_filter_conv0_q, scale_filter_conv0, 0)
Bias_conv0 = Quantization2d(Bias_conv0, min_bias_conv0_q, max_bias_conv0_q, scale_bias_conv0, 0)
Conv1 = Quantization2d(Conv1, min_filter_conv1_q, max_filter_conv1_q, scale_filter_conv1, 0)
Bias_conv1 = Quantization2d(Bias_conv1, min_bias_conv1_q, max_bias_conv1_q, scale_bias_conv1, 0)

W0 = Quantization2d(W0, min_w0_q, max_w0_q, scale_w0, 0)
W1 = Quantization2d(W1, min_w1_q, max_w1_q, scale_w1, 0)
W2 = Quantization2d(W2, min_w2_q, max_w2_q, scale_w2, 0)

B0 = Quantization2d(B0, min_b0_q, max_b0_q, scale_b0, 0)
B1 = Quantization2d(B1, min_b1_q, max_b1_q, scale_b1, 0)
B2 = Quantization2d(B2, min_b2_q, max_b2_q, scale_b2, 0)

In [60]:
quatized_image_temp = Quantization3d(image_test[1], min_input_q, max_input_q, scale_input, 0)

quatized_image = quatized_image_temp[:, :, 0]

In [61]:
print(quatized_image.shape)

(32, 32)


In [62]:
create_txt_file_for_input_verilog(quatized_image, bit_width_image, image_path_verilog)

create_txt_file_for_input_verilog(quatized_image, bit_width_filter_conv0, Conv0_path_verilog)
create_txt_file_for_input_verilog(quatized_image, bit_width_bias_conv0, Bias_conv0_path_verilog)
create_txt_file_for_input_verilog(quatized_image, bit_width_filter_conv1, Conv1_path_verilog)
create_txt_file_for_input_verilog(quatized_image, bit_width_bias_conv1, Bias_conv1_path_verilog)

In [16]:
a = np.random.randint(0, 5, (2, 2, 3, 2))
print(a)
print('========')
print(a[1])

[[[[4 2]
   [3 4]
   [4 1]]

  [[2 0]
   [0 0]
   [0 0]]]


 [[[4 1]
   [3 2]
   [4 3]]

  [[2 0]
   [1 1]
   [0 0]]]]
[[[4 1]
  [3 2]
  [4 3]]

 [[2 0]
  [1 1]
  [0 0]]]


In [8]:
b = a.reshape(-1, 1)
print(b)

[[3]
 [2]
 [0]
 [1]
 [4]
 [4]
 [4]
 [3]
 [2]
 [0]
 [4]
 [3]
 [1]
 [3]
 [3]
 [4]
 [2]
 [1]
 [0]
 [2]
 [2]
 [0]
 [3]
 [2]]
